In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

In [ ]:
rapm = pd.read_csv('rapm97-14.csv').set_index('Name')
#Ovr/100

In [ ]:
WS_REAL = defaultdict(list)
COLS = ['MP','FG','FGA','FT','FTA','TRB','AST','PF','PTS']
for YR in range(1997,2014):
    table = np.load('tables_{}.pkl'.format(YR))
    for team in table:
        stats = table[team]['per_minute'][COLS].replace('',0)
        stats['PTS'] -= table[team]['per_minute']['3P'].replace('',0)
        for item in stats.itertuples():
            WS_REAL[item[0].lower()].append(item[1:])

In [ ]:
for k,v in WS_REAL.items():
    WS_REAL[k] = np.array([[0 if e == '' else e for e in _] for _ in v])#.mean(0)

In [ ]:
valid_names = [_.lower() for _ in rapm.index if type(_) == str]

In [ ]:
X = []
y = []
Xmean = []
for row in rapm.itertuples():
    if not type(row[0]) == str:
        continue
    name = row[0].lower()
    if name in WS_REAL:
        X.append(WS_REAL[name])
        Xmean.append(WS_REAL[name].mean(0))
        y.append([row[-1] for _ in range(len(WS_REAL[name]))])

In [ ]:
X = np.vstack(X)
y = np.hstack(y)
X.shape,y.shape

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection, model_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=False)
scalerX = preprocessing.StandardScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
#prescale_y = scalery.fit_transform(np.array(y).reshape((-1,1)))
prescale_X = fexp.fit_transform(X)
X_train, X_test, y_train, y_test = model_selection.train_test_split(prescale_X, y, test_size=0.25, random_state=42)

In [ ]:
clf = linear_model.Ridge(1,fit_intercept=False)
clf.fit(X_train,y_train)
clf.score(X_train,y_train),clf.score(X_test,y_test)

In [ ]:
col_names = fexp.get_feature_names(COLS)

coeffs = clf.coef_ 
v = np.argsort(abs(coeffs))[::-1]
coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v]
#for v,n in sorted(coeffs2,reverse=True):
#    print('{:.2f} * {} + '.format(v,n),end='')
print('| Var      | Coeff |')
print('|----------|-------|')
for v,n in sorted(coeffs2,reverse=True):
    print('|{:10s}|{:.2f}|'.format(n,v))
#for v,n in sorted(coeffs2,reverse=True):
#    print('\t{:25s}\t{:.2f}'.format(n,v))


In [ ]:
coeffs2

In [ ]:
plt.hist(y,30,density=True)
plt.hist(clf.predict(prescale_X),30,density=True)

In [ ]:
for i,v in enumerate(clf.predict(fexp.transform(Xmean))):
    print('{:20s}\t{:2f}\t{:2f}'.format(rapm.index[i],v,rapm.iloc[i,-1]))

In [ ]:
clf.score(prescale_X,y)